In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/faceforensics/FF++/fake/02_13__exit_phone_room__CP5HFV3K.mp4
/kaggle/input/faceforensics/FF++/fake/03_14__talking_against_wall__ZC2KYASW.mp4
/kaggle/input/faceforensics/FF++/fake/03_15__outside_talking_pan_laughing__Y11NT1YX.mp4
/kaggle/input/faceforensics/FF++/fake/07_26__walking_down_street_outside_angry__FGNGC2GT.mp4
/kaggle/input/faceforensics/FF++/fake/07_03__hugging_happy__7NGMD8FT.mp4
/kaggle/input/faceforensics/FF++/fake/07_09__walk_down_hall_angry__N9CWME71.mp4
/kaggle/input/faceforensics/FF++/fake/01_12__outside_talking_pan_laughing__TNI7KUZ6.mp4
/kaggle/input/faceforensics/FF++/fake/07_03__podium_speech_happy__6PHZRQ4H.mp4
/kaggle/input/faceforensics/FF++/fake/07_02__walking_down_street_outside_angry__O4SXNLRL.mp4
/kaggle/input/faceforensics/FF++/fake/08_05__walk_down_hall_angry__FBICSP2C.mp4
/kaggle/input/faceforensics/FF++/fake/06_18__outside_talking_pan_laughing__DEA1TCLN.mp4
/kaggle/input/faceforensics/FF++/fake/01_03__talking_against_wall__JZUXXFRB.mp4
/ka

In [ ]:
import torch
print("GPU available:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

# Essential libraries
import os
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt

# Deep learning
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

# Face detection
!pip install facenet-pytorch --quiet
from facenet_pytorch import MTCNN


GPU available: True
Device: Tesla T4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 25.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 90.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 99.6 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 78.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 29.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 13.5 MB/s eta 0:

In [ ]:
#Directory setup and face detector
from facenet_pytorch import MTCNN
import torchvision.transforms as T
from PIL import Image
import glob

# Define paths to real and fake videos
REAL_VID_DIR = '/kaggle/input/faceforensics/FF++/real'
FAKE_VID_DIR = '/kaggle/input/faceforensics/FF++/fake'

# Output directory for extracted faces
FACE_SAVE_DIR = '/kaggle/working/extracted_faces'
os.makedirs(FACE_SAVE_DIR, exist_ok=True)

# Initialize MTCNN
device = 'cuda' if torch.cuda.is_available() else 'cpu'
mtcnn = MTCNN(image_size=160, margin=20, device=device, post_process=True)


In [ ]:
#Extract 10 aligned faces from each video
import cv2
import torch
import torchvision.transforms as T
from facenet_pytorch import MTCNN
import os
import numpy as np
from tqdm import tqdm


mtcnn = MTCNN(image_size=224, margin=20, device=device, post_process=True)
FACE_SAVE_DIR = '/kaggle/working/extracted_faces'
os.makedirs(FACE_SAVE_DIR, exist_ok=True)

# Function to extract faces from a video
def extract_faces_from_video(video_path, save_path, max_faces=10, target_size=(224, 224)):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = np.linspace(0, total_frames - 1, num=max_faces, dtype=int)

    for idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        success, frame = cap.read()
        if not success:
            continue

        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face = mtcnn(rgb)

        if face is not None:
            frames.append(face)

    cap.release()

    if len(frames) >= 1:
        while len(frames) < max_faces:
            frames.append(frames[-1].clone())
        faces_tensor = torch.stack(frames[:max_faces])  # (T, C, H, W)
        torch.save(faces_tensor, save_path)
    else:
        print(f"⚠️ No face detected in {os.path.basename(video_path)}")

# Batch extract from all videos
def extract_all_faces(input_dir, label):
    video_paths = glob.glob(os.path.join(input_dir, '*.mp4'))
    print(f"[{label.upper()}] Found {len(video_paths)} videos.")

    for video_path in tqdm(video_paths):
        vid_name = os.path.basename(video_path).replace('.mp4', '')
        save_path = os.path.join(FACE_SAVE_DIR, f'{label}_{vid_name}.pt')

        if not os.path.exists(save_path):
            extract_faces_from_video(video_path, save_path)




In [ ]:
# Run for re-extracting the faces
#!rm -rf /kaggle/working/extracted_faces


In [5]:
# Run face extraction for both classes
extract_all_faces(REAL_VID_DIR, label='real')
extract_all_faces(FAKE_VID_DIR, label='fake')


[REAL] Found 200 videos.


100%|██████████| 200/200 [17:20<00:00,  5.20s/it]


[FAKE] Found 200 videos.


100%|██████████| 200/200 [16:41<00:00,  5.01s/it]


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision.transforms as T
import os
import glob
from sklearn.model_selection import train_test_split

# Dataset Loader (updated for 10 faces, 224x224)

import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split

class DeepfakeDataset(Dataset):
    def __init__(self, face_dir, transform=None):
        self.face_paths = glob.glob(os.path.join(face_dir, '*.pt'))
        self.transform = transform

    def __len__(self):
        return len(self.face_paths)

    def __getitem__(self, idx):
        face_path = self.face_paths[idx]
        faces = torch.load(face_path)  # shape: (T, 3, 224, 224)
    
        if self.transform:
            faces = torch.stack([self.transform(face) for face in faces])
    
        label = 1 if 'fake' in os.path.basename(face_path) else 0
        return faces, torch.tensor(label, dtype=torch.float32)



transform = T.Compose([
    T.ToPILImage(),
    T.RandomHorizontalFlip(),
    T.RandomRotation(degrees=15),
    T.ColorJitter(brightness=0.3, contrast=0.3),
    T.ToTensor(),
    T.Normalize([0.5]*3, [0.5]*3)
])


full_dataset = DeepfakeDataset(FACE_SAVE_DIR, transform=transform)
train_idx, val_idx = train_test_split(range(len(full_dataset)), test_size=0.2, random_state=42)

train_dataset = Subset(full_dataset, train_idx)
val_dataset   = Subset(full_dataset, val_idx)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=2)


In [ ]:
import torch
import torch.nn as nn
import torch.utils.checkpoint as cp
import timm


class XceptionBackbone(nn.Module):
    def __init__(self, freeze_stages=True):
        super().__init__()
        self.backbone = timm.create_model(
            'xception', pretrained=True, features_only=True
        )
        self.out_channels = self.backbone.feature_info[-1]['num_chs']  # usually 2048

        if freeze_stages:
            for param in self.backbone.parameters():
                param.requires_grad = False

           
            for name, module in list(self.backbone.named_children())[-2:]: 
                for param in module.parameters():
                    param.requires_grad = True

    def forward(self, x):
        features = self.backbone(x)  # List of feature maps
        return features[-1]          # Final feature map (B*T, 2048, H, W)


class PatchTokenizer(nn.Module):
    def __init__(self, input_channels=2048, embed_dim=2048):
        super().__init__()
        self.C_in = input_channels
        self.C_out = embed_dim

        self.proj = nn.Linear(self.C_in, self.C_out) if self.C_in != self.C_out else nn.Identity()
        self.spatial_cls = nn.Parameter(torch.randn(1, 1, self.C_out))
        self.temporal_cls = nn.Parameter(torch.randn(1, 1, self.C_out))
        self.pos_embed = nn.Parameter(
            torch.randn(1, num_frames + 1, patch_tokens + 1, self.C_out)
        )  # (1, T+1, HW+1, C)

    def forward(self, x):  # (B, T, C, H, W)
        B, T, C, H, W = x.shape
        HW = H * W
        x = x.view(B, T, C, HW).permute(0, 1, 3, 2)  # (B, T, HW, C)
        x = self.proj(x)

        spatial_cls = self.spatial_cls.expand(B, T, 1, -1)
        x = torch.cat([spatial_cls, x], dim=2)  # (B, T, HW+1, C)

        temporal_cls = self.temporal_cls.expand(B, 1, x.size(2), -1)
        x = torch.cat([temporal_cls, x], dim=1)  # (B, T+1, HW+1, C)

        pos_embed = torch.randn(T + 1, x.size(2), self.C_out, device=x.device)
        x = x + self.pos_embed  # (B, T+1, HW+1, C) with learnable embeddings
        return x


class DecomposedSTBlock(nn.Module):
    def __init__(self, dim, num_heads):
        super().__init__()
        self.spatial_attn = nn.MultiheadAttention(embed_dim=dim, num_heads=num_heads, batch_first=True)
        self.temporal_attn = nn.MultiheadAttention(embed_dim=dim, num_heads=num_heads, batch_first=True)
        self.norm1 = nn.LayerNorm(dim)
        self.norm2 = nn.LayerNorm(dim)

    def forward(self, x):
        return cp.checkpoint(self._forward_impl, x)

    def _forward_impl(self, x):  # (B, T+1, HW+1, C)
        B, T1, P1, C = x.shape
        spatial_out = []
        for t in range(T1):
            x_t = x[:, t]  # (B, P, C)
            q = k = self._self_subtract(x_t)
            out, _ = self.spatial_attn(q, k, x_t)
            spatial_out.append(self.norm1(x_t + out))
        x = torch.stack(spatial_out, dim=1)  # (B, T+1, P, C)

        x = x.permute(0, 2, 1, 3)  # (B, P, T+1, C)
        temporal_out = []
        for p in range(P1):
            x_p = x[:, p]  # (B, T+1, C)
            q = k = self._self_subtract(x_p)
            out, _ = self.temporal_attn(q, k, x_p)
            temporal_out.append(self.norm2(x_p + out))
        x = torch.stack(temporal_out, dim=1)  # (B, P, T+1, C)
        x = x.permute(0, 2, 1, 3)  # (B, T+1, P, C)
        return x

    def _self_subtract(self, x):
        return x - x.mean(dim=-1, keepdim=True)


class ISTVT(nn.Module):
    def __init__(self, num_frames=10, embed_dim=2048, depth=2, num_heads=4):
        super().__init__()
        self.num_frames = num_frames
        self.feature_extractor = XceptionBackbone()
        self.tokenizer = None

        self.encoder_blocks = nn.Sequential(*[
            DecomposedSTBlock(embed_dim, num_heads) for _ in range(depth)
        ])

        self.classifier = nn.Sequential(
            nn.LayerNorm(embed_dim),
            nn.Linear(embed_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 1)
        )

    def forward(self, x):  # x: (B, T, 3, 299, 299)
        B, T, C, H, W = x.shape
        x = x.view(B * T, C, H, W)

        feats = self.feature_extractor(x)  # (B*T, C_feat, H_feat, W_feat)
        _, C_feat, H_feat, W_feat = feats.shape
        feats = feats.view(B, T, C_feat, H_feat, W_feat)

        if self.tokenizer is None:
            self.tokenizer = PatchTokenizer(input_channels=C_feat).to(x.device)

        tokens = self.tokenizer(feats)               # (B, T+1, HW+1, C)
        encoded = self.encoder_blocks(tokens)        # (B, T+1, HW+1, C)
        cls_token = encoded[:, 0, 0]                 # (B, C)
        logits = self.classifier(cls_token)          # (B, 1)
        return logits


In [ ]:
model = ISTVT(num_frames=10).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
def compute_accuracy(logits, labels):
    preds = (torch.sigmoid(logits) > 0.5).float()
    return (preds == labels).float().mean().item()

def train_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss, total_acc = 0.0, 0.0

    for x, y in tqdm(loader, desc='Training'):
        x, y = x.to(device), y.float().to(device)

        optimizer.zero_grad()
        outputs = model(x)  # shape: [B, 1]
        loss = criterion(outputs.squeeze(1), y)
        acc = compute_accuracy(outputs.squeeze(1), y)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_acc += acc

    return total_loss / len(loader), total_acc / len(loader)

def validate(model, loader, criterion):
    model.eval()
    val_loss, val_acc = 0.0, 0.0

    with torch.no_grad():
        for x, y in tqdm(loader, desc='Validating'):
            x, y = x.to(device), y.float().to(device)
            outputs = model(x)
            loss = criterion(outputs.squeeze(1), y)
            acc = compute_accuracy(outputs.squeeze(1), y)

            val_loss += loss.item()
            val_acc += acc

    return val_loss / len(loader), val_acc / len(loader)


In [ ]:
num_epochs = 15
best_val_loss = float('inf')

for epoch in range(num_epochs):
    print(f"\n🌟 Epoch {epoch+1}/{num_epochs}")

    train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_acc = validate(model, val_loader, criterion)

    # Step scheduler if used
    scheduler.step(val_loss)

    # Show LR
    for param_group in optimizer.param_groups:
        print(f"🔁 Learning Rate: {param_group['lr']}")

    # Show metrics
    print(f"📈 Train Loss: {train_loss:.4f} | Accuracy: {train_acc:.4f}")
    print(f"📉 Val   Loss: {val_loss:.4f} | Accuracy: {val_acc:.4f}")

    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_istvt.pth")
        print("✅ Best model saved!")



🌟 Epoch 1/15


Validating: 100%|██████████| 20/20 [00:02<00:00,  8.26it/s]


🔁 Learning Rate: 0.0001
📈 Train Loss: 0.6760 | Accuracy: 0.5563
📉 Val   Loss: 0.6468 | Accuracy: 0.6000
✅ Best model saved!

🌟 Epoch 2/15


Validating: 100%|██████████| 20/20 [00:02<00:00,  7.90it/s]


🔁 Learning Rate: 0.0001
📈 Train Loss: 0.5247 | Accuracy: 0.7406
📉 Val   Loss: 0.5034 | Accuracy: 0.7750
✅ Best model saved!

🌟 Epoch 3/15


...

🔁 Learning Rate: 5e-05
📈 Train Loss: 0.0792 | Accuracy: 0.9812
📉 Val   Loss: 0.2214 | Accuracy: 0.8875
✅ Best model saved!

🌟 Epoch 14/15


Validating: 100%|██████████| 20/20 [00:02<00:00,  8.45it/s]


🔁 Learning Rate: 5e-05
📈 Train Loss: 0.0773 | Accuracy: 0.9656
📉 Val   Loss: 0.2450 | Accuracy: 0.8625

🌟 Epoch 15/15


Validating: 100%|██████████| 20/20 [00:02<00:00,  8.54it/s]

🔁 Learning Rate: 5e-05
📈 Train Loss: 0.0811 | Accuracy: 0.9688
📉 Val   Loss: 0.4450 | Accuracy: 0.8250
